In [ ]:
#Install packages

! pip install transformers datasets
! pip install torch
! pip install tensorflow
! pip install tika
! pip install nltk

from transformers import pipeline
import os
import re
import math
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

nltk.download('all')

Clone GitHub repo with all of the data

In [ ]:
!git clone https://github.com/bcarter24/testRepo.git

Move to data directory

In [ ]:
%cd /content/testRepo/data/US\ Federal\ Code\ of\ Law/Cleaner_ungrouped/

In [ ]:
directory1 = "./Title18_Chapter44/"
directory2 = "./Title26_Chapter53/"

document_array = []
biggestSentArr = 0
secondLargest = 0
thirdLargest = 0

#enumerate files in the directory into list for directory 1
files_in_direc1 = os.listdir(directory1)
#print(files_in_direc1)
for file in files_in_direc1:
  #check if file is valid
  if os.path.isfile(os.path.join(directory1, file)):
    #open file to read
    file_handle = open(os.path.join(directory1, file), 'r+')
    this_doc = str( file_handle.read())
    file_handle.close()
    #get rid of section part at the beginning but not section references later on
    #this_doc = re.sub(r"§\s478\.", "", this_doc)
    #this_doc = re.sub(r"§", "", this_doc)
    
    #tokenize
    sentence_array = word_tokenize(this_doc,language='english', preserve_line=True)
    document_array.append(sentence_array)
    print(sentence_array)
    print('length of sentence array is: ' + str(len(sentence_array)) + '\n')
    if len(sentence_array) > biggestSentArr :
      thirdLargest = secondLargest
      secondLargest = biggestSentArr
      biggestSentArr = len(sentence_array)
    #print('current length of Doc array is: ' + str(len(document_array)))

print('\nThere are ' + str(len(document_array)) + ' documents in the document_array')
print('The largest Sentence Array so far has a token size of ' + str(biggestSentArr) + ' tokens.')
print('The second largest Sentence Array so far has a token size of ' + str(secondLargest) + ' tokens.')
print('The third largest Sentence Array so far has a token size of ' + str(thirdLargest) + ' tokens.\n\n\n')


#repeat for directory 2
files_in_direc1 = os.listdir(directory2)
#print(files_in_direc1)
for file in files_in_direc1:
  #check if file is valid
  if os.path.isfile(os.path.join(directory2, file)):
    #open file to read
    file_handle = open(os.path.join(directory2, file), 'r+')
    this_doc = str( file_handle.read())
    file_handle.close()
    #get rid of section part at the beginning but not section references later on
    #this_doc = re.sub(r"§\s478\.", "", this_doc)
    #this_doc = re.sub(r"§", "", this_doc)
    
    #tokenize
    sentence_array = word_tokenize(this_doc,language='english', preserve_line=True)
    document_array.append(sentence_array)
    print(sentence_array)
    print('length of sentence array is: ' + str(len(sentence_array)) + '\n')
    if len(sentence_array) > biggestSentArr :
      thirdLargest = secondLargest
      secondLargest = biggestSentArr
      biggestSentArr = len(sentence_array)
    #print('current length of Doc array is: ' + str(len(document_array)))

print('\nThere are ' + str(len(document_array)) + ' documents in the document_array')
print('The largest Sentence Array so far has a token size of ' + str(biggestSentArr) + ' tokens.')
print('The second largest Sentence Array so far has a token size of ' + str(secondLargest) + ' tokens.')
print('The third largest Sentence Array so far has a token size of ' + str(thirdLargest) + ' tokens.\n\n\n')